---
### <center> Carga de Datos

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from scipy import stats
from scipy.stats import kendalltau
from sklearn.preprocessing import StandardScaler, LabelEncoder


In [2]:
# Carga y visualización parcial de Datos:
df_ori = pd.read_csv('datasets/demanda-ltimos-aos.csv')
df_ori.head(5)

,id,anio,mes,agente_nemo,agente_descripcion,tipo_agente,region,provincia,categoria_area,categoria_demanda,tarifa,categoria_tarifa,demanda_MWh,fecha_proceso,lote_id_log,indice_tiempo
0,699232,2017,1,AARGTAOY,AEROP ARG 2000 - Aeroparque,GU,GRAN BS.AS.,BUENOS AIRES,Gran Usuario MEM,Gran Usuario,GUMAS/AUTOGENERADORES,Industrial/Comercial Grande,1990.439,2020-05-05 11:06:49.000713,67,2017-01
1,699233,2017,1,ABRILHCY,ABRIL CLUB DE CAMPO,GU,GRAN BS.AS.,BUENOS AIRES,Gran Usuario MEM,Gran Usuario,GUMAS/AUTOGENERADORES,Industrial/Comercial Grande,1609.464,2020-05-05 11:06:49.000713,67,2017-01
2,699234,2017,1,ACARQQ3Y,ASOC.COOP.ARG. - Quequén,GU,BUENOS AIRES,BUENOS AIRES,Gran Usuario MEM,Gran Usuario,GUMAS/AUTOGENERADORES,Industrial/Comercial Grande,421.334,2020-05-05 11:06:49.000713,67,2017-01
3,699235,2017,1,ACARSLSY,ASOC.COOP.ARG. - San Lorenzo,GU,LITORAL,SANTA FE,Gran Usuario MEM,Gran Usuario,GUMAS/AUTOGENERADORES,Industrial/Comercial Grande,449.145,2020-05-05 11:06:49.000713,67,2017-01
4,699236,2017,1,ACERBR1Y,Planta Bragado,GU,BUENOS AIRES,BUENOS AIRES,Gran Usuario MEM,Gran Usuario,GUMAS/AUTOGENERADORES,Industrial/Comercial Grande,18073.170,2020-05-05 11:06:49.000713,67,2017-01


---
### <center> Preparación de Datos

In [3]:
df_ori.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40388 entries, 0 to 40387
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  40388 non-null  int64  
 1   anio                40388 non-null  int64  
 2   mes                 40388 non-null  int64  
 3   agente_nemo         40388 non-null  object 
 4   agente_descripcion  40388 non-null  object 
 5   tipo_agente         40388 non-null  object 
 6   region              40388 non-null  object 
 7   provincia           40388 non-null  object 
 8   categoria_area      40388 non-null  object 
 9   categoria_demanda   40388 non-null  object 
 10  tarifa              40388 non-null  object 
 11  categoria_tarifa    40388 non-null  object 
 12  demanda_MWh         40388 non-null  float64
 13  fecha_proceso       40388 non-null  object 
 14  lote_id_log         40388 non-null  int64  
 15  indice_tiempo       40388 non-null  object 
dtypes: f

In [4]:
# Chequeo de registros duplicados:
df_ori.duplicated().value_counts()

False    40388
dtype: int64

In [5]:
# Chequeo de registros negativos
condicion_negativos = df_ori['demanda_MWh'] < 0
filas_negativas = df_ori[condicion_negativos]
filas_negativas

,id,anio,mes,agente_nemo,agente_descripcion,tipo_agente,region,provincia,categoria_area,categoria_demanda,tarifa,categoria_tarifa,demanda_MWh,fecha_proceso,lote_id_log,indice_tiempo
34,699266,2017,1,APELPALD,APELP,DI,COMAHUE,LA PAMPA,Resto,Distribuidor,MERCADO TERMINO DISTRIB,Mercado a Término,-0.001,2020-05-05 11:06:49.000713,67,2017-01
94,699325,2017,1,C16OCTUW,COOP. 16 DE OCTUBRE,DI,PATAGONICA,CHUBUT,Resto,Distribuidor,MERCADO TERMINO DISTRIB,Mercado a Término,-0.001,2020-05-05 11:06:49.000713,67,2017-01
103,699334,2017,1,C3AR3A3W,COOP. CELTA - TRES ARROYOS,DI,BUENOS AIRES,BUENOS AIRES,Resto,Distribuidor,MERCADO TERMINO DISTRIB,Mercado a Término,-0.001,2020-05-05 11:06:49.000713,67,2017-01
115,699346,2017,1,CALFAVQW,CALF NEUQUEN DISTRIBUIDOR,DI,COMAHUE,NEUQUEN,Resto,Distribuidor,MERCADO TERMINO DISTRIB,Mercado a Término,-0.001,2020-05-05 11:06:49.000713,67,2017-01
151,699382,2017,1,CBARKE3W,COOPERATIVA DE BARKER,DI,BUENOS AIRES,BUENOS AIRES,Resto,Distribuidor,MERCADO TERMINO DISTRIB,Mercado a Término,-0.001,2020-05-05 11:06:49.000713,67,2017-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,739227,2020,2,EDERSARD,EMP DE ENERGIA DE RIO NEGRO SA,DI,COMAHUE,RIO NEGRO,Resto,Distribuidor,MERCADO TERMINO DISTRIB,Mercado a Término,-0.001,2020-05-05 11:06:49.000713,67,2020-02
40011,739243,2020,2,EDESASAD,EMP.DIST.ENERGIA DE SALTA,DI,NOROESTE,SALTA,Resto,Distribuidor,MERCADO TERMINO DISTRIB,Mercado a Término,-0.001,2020-05-05 11:06:49.000713,67,2020-02
40066,739298,2020,2,EPESAFSD,EPESF DISTRIBUIDOR,DI,LITORAL,SANTA FE,Santa Fe,Distribuidor,MERCADO TERMINO DISTRIB,Mercado a Término,-0.001,2020-05-05 11:06:49.000713,67,2020-02
40233,739465,2020,2,SECHEPHD,SECHEEP,DI,NORESTE,CHACO,Resto,Distribuidor,MERCADO TERMINO DISTRIB,Mercado a Término,-0.001,2020-05-05 11:06:49.000713,67,2020-02


In [ ]:
# Filtrar las filas negativas
# Condición para seleccionar no incluir los negativos 
condicion = (df_ori['demanda_MWh'] >= 0) 

# Reiniciar el índice de la serie booleana
#condicion = condicion.reset_index(drop=True)

df_ori = df_ori.loc[condicion]

Observamos que el DataFrame posee 39624 registros, y que cada columna posee la misma cantidad de objetos no nulos (ni *'Nan'* ni *'None'*) a excepción de * demandas negativas * del Mercado a término, que tampoco los incluímos. Además, no existen registros duplicados.

In [6]:
# Chequeo de países presentes:
df_ori['provincia'].value_counts()

BUENOS AIRES      22056
CHUBUT             2154
SANTA FE           2041
MENDOZA            1917
ENTRE RIOS         1299
NEUQUEN            1211
RIO NEGRO          1023
CORDOBA             960
SAN JUAN            908
SANTA CRUZ          847
TUCUMAN             830
SAN LUIS            661
LA RIOJA            521
MISIONES            513
CORRIENTES          510
SALTA               478
SGO.DEL ESTERO      463
JUJUY               462
CATAMARCA           439
CHACO               401
LA PAMPA            359
FORMOSA             335
Name: provincia, dtype: int64

In [8]:
df_ori['anio'].value_counts()

2017    14713
2018    14304
2019     9810
2020     1561
Name: anio, dtype: int64

In [9]:
# Condición para seleccionar provincia = 'CORDOBA' y años 2017, 2018, 2019 
condicion = (df_ori['provincia'].str.contains('BUENOS AIRES')) & (df_ori['anio'].isin([2017, 2018, 2019]) & (df_ori['demanda_MWh'] >= 0) ) 

df_ori = df_ori.loc[condicion]

,id,anio,mes,agente_nemo,agente_descripcion,tipo_agente,region,provincia,categoria_area,categoria_demanda,tarifa,categoria_tarifa,demanda_MWh,fecha_proceso,lote_id_log,indice_tiempo
0,699232,2017,1,AARGTAOY,AEROP ARG 2000 - Aeroparque,GU,GRAN BS.AS.,BUENOS AIRES,Gran Usuario MEM,Gran Usuario,GUMAS/AUTOGENERADORES,Industrial/Comercial Grande,1990.439,2020-05-05 11:06:49.000713,67,2017-01
1,699233,2017,1,ABRILHCY,ABRIL CLUB DE CAMPO,GU,GRAN BS.AS.,BUENOS AIRES,Gran Usuario MEM,Gran Usuario,GUMAS/AUTOGENERADORES,Industrial/Comercial Grande,1609.464,2020-05-05 11:06:49.000713,67,2017-01
2,699234,2017,1,ACARQQ3Y,ASOC.COOP.ARG. - Quequén,GU,BUENOS AIRES,BUENOS AIRES,Gran Usuario MEM,Gran Usuario,GUMAS/AUTOGENERADORES,Industrial/Comercial Grande,421.334,2020-05-05 11:06:49.000713,67,2017-01
4,699236,2017,1,ACERBR1Y,Planta Bragado,GU,BUENOS AIRES,BUENOS AIRES,Gran Usuario MEM,Gran Usuario,GUMAS/AUTOGENERADORES,Industrial/Comercial Grande,18073.170,2020-05-05 11:06:49.000713,67,2017-01
6,699238,2017,1,ACINTBOY,ACINDAR PTA. TABLADA,GU,GRAN BS.AS.,BUENOS AIRES,Gran Usuario MEM,Gran Usuario,GUMAS/AUTOGENERADORES,Industrial/Comercial Grande,2017.775,2020-05-05 11:06:49.000713,67,2017-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38820,738048,2019,12,ADBACACY,ADMINISTRACION DBA SAS,GU,GRAN BS.AS.,BUENOS AIRES,Gran Usuario MEM,Gran Usuario,GUMAS/AUTOGENERADORES,Industrial/Comercial Grande,246.056,2020-05-05 11:06:49.000713,67,2019-12
38824,738052,2019,12,CARHBB2Y,BOORMALT ARGENTINA S.A.S,GU,BUENOS AIRES,BUENOS AIRES,Gran Usuario MEM,Gran Usuario,GUMAS/AUTOGENERADORES,Industrial/Comercial Grande,1522.088,2020-05-05 11:06:49.000713,67,2019-12
38825,738053,2019,12,CERV3A3Y,CERVECERIA Y MALTERIA QUILMES,GU,BUENOS AIRES,BUENOS AIRES,Gran Usuario MEM,Gran Usuario,GUMAS/AUTOGENERADORES,Industrial/Comercial Grande,2373.019,2020-05-05 11:06:49.000713,67,2019-12
38826,738054,2019,12,FARGPIOY,COMPAÑIA DE ALIMENTOS FARGO,GU,GRAN BS.AS.,BUENOS AIRES,Gran Usuario MEM,Gran Usuario,GUMAS/AUTOGENERADORES,Industrial/Comercial Grande,833.076,2020-05-05 11:06:49.000713,67,2019-12


In [10]:
df_ori.describe()

,id,anio,mes,demanda_MWh,lote_id_log
count,20829.000000,20829.000000,20829.000000,2.082900e+04,20829.0
mean,718557.021508,2017.871477,6.468145,9.321066e+03,67.0
std,11181.171737,0.782556,3.464338,5.373789e+04,0.0
min,699232.000000,2017.000000,1.000000,0.000000e+00,67.0
25%,708966.000000,2017.000000,3.000000,2.042860e+02,67.0
50%,718585.000000,2018.000000,6.000000,9.265180e+02,67.0
75%,728253.000000,2019.000000,9.000000,2.740801e+03,67.0
max,738055.000000,2019.000000,12.000000,1.572308e+06,67.0


In [12]:
# Verificar los valores nulos en cada columna:
valores_nulos = df_ori.isnull()

# Contar los valores nulos en cada columna
conteo_nulos = valores_nulos.sum()

# Muestra el conteo de valores nulos
print(conteo_nulos)


id                    0
anio                  0
mes                   0
agente_nemo           0
agente_descripcion    0
tipo_agente           0
region                0
provincia             0
categoria_area        0
categoria_demanda     0
tarifa                0
categoria_tarifa      0
demanda_MWh           0
fecha_proceso         0
lote_id_log           0
indice_tiempo         0
dtype: int64


In [13]:
# Información actualizada:
df_ori.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20829 entries, 0 to 38827
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  20829 non-null  int64  
 1   anio                20829 non-null  int64  
 2   mes                 20829 non-null  int64  
 3   agente_nemo         20829 non-null  object 
 4   agente_descripcion  20829 non-null  object 
 5   tipo_agente         20829 non-null  object 
 6   region              20829 non-null  object 
 7   provincia           20829 non-null  object 
 8   categoria_area      20829 non-null  object 
 9   categoria_demanda   20829 non-null  object 
 10  tarifa              20829 non-null  object 
 11  categoria_tarifa    20829 non-null  object 
 12  demanda_MWh         20829 non-null  float64
 13  fecha_proceso       20829 non-null  object 
 14  lote_id_log         20829 non-null  int64  
 15  indice_tiempo       20829 non-null  object 
dtypes: f

El DataFrame consta de 907 registros, sin inconsistencias, valores no nulos ni duplicados. Todos los registros son pertenecientes a Córdoba y demanda de 2017, 2018, 2019

---
### <center> Análisis exploratorio de Datos

Al consultar las **'categoria_tarifa'**, observamos que existen varias clasificaciones para las tarifas (Residencial, Comercial, Mercado a término e Industrial/Comercial Grande):

In [14]:
df_ori['categoria_tarifa'].value_counts()

Industrial/Comercial Grande    11176
Residencial                     8247
Comercial                       1404
Mercado a Término                  2
Name: categoria_tarifa, dtype: int64

In [15]:
# Generar nuevo dataframe con columnas de interés
df = df_ori.iloc[:,[1,2,11,12]]
df = df.rename(columns={'anio': 'anio',
                         'mes': 'mes',
            'categoria_tarifa':'categoria_tarifa',
                 'demanda_MWh': 'demanda_MWh'})

In [16]:
df

,anio,mes,categoria_tarifa,demanda_MWh
0,2017,1,Industrial/Comercial Grande,1990.439
1,2017,1,Industrial/Comercial Grande,1609.464
2,2017,1,Industrial/Comercial Grande,421.334
4,2017,1,Industrial/Comercial Grande,18073.170
6,2017,1,Industrial/Comercial Grande,2017.775
...,...,...,...,...
38820,2019,12,Industrial/Comercial Grande,246.056
38824,2019,12,Industrial/Comercial Grande,1522.088
38825,2019,12,Industrial/Comercial Grande,2373.019
38826,2019,12,Industrial/Comercial Grande,833.076


In [42]:
# ###########################

In [59]:
# Diccionario para almacenar los datos agrupados
data = {
    'tarifa': [],
    'cantidad': [],
    'energia': [],
    'total': 0  # Inicializar 'total' en 0
}
total_general = 0
# Agrupar el DataFrame por 'categoria_tarifa' y calcular la suma de 'demanda_MWh' para cada grupo
grupo_tarifas = df.groupby('categoria_tarifa')

for tarifa, grupo in grupo_tarifas:
    cantidad_registros = len(grupo)
    demanda_energia = (grupo['demanda_MWh'].sum())/1000

    # Agregar los valores al diccionario
    data['tarifa'].append(tarifa)
    data['cantidad'].append(cantidad_registros)
    data['energia'].append(demanda_energia)
    data['total'] += demanda_energia  # Actualizar el 'total' general
    total_general += demanda_energia
    
# Agregar el total general al diccionario 'data'
tari_tot = 'TOT'
cant_tot = len(df)

# Diccionario para el total general
total_general = {
    'tarifa': tari_tot,
    'cantidad': cant_tot,
    'energia': 0,
    'total': 0  # El total general es igual a la energía total
}

# Agregar el total general al diccionario 'data'
data['tarifa'].append(total_general['tarifa'])
data['cantidad'].append(total_general['cantidad'])
data['energia'].append(total_general['energia'])
data['total'] += total_general['total']

# Convertir el diccionario en un DataFrame
data_agrupada = pd.DataFrame(data)

# Mostrar el DataFrame resultante
print(data_agrupada)



                        tarifa  cantidad       energia         total
0                    Comercial      1404  5.311848e+04  1.943426e+08
1  Industrial/Comercial Grande     11176  5.573125e+04  1.943426e+08
2            Mercado a Término         2  1.979000e-03  1.943426e+08
3                  Residencial      8247  8.529875e+04  1.943426e+08
4                          TOT     20829  1.941485e+08  1.943426e+08


In [64]:
# Calculamos matriz de correlacion de todo el conjunto de datos
matriz_correlacion = data_agrupada[['cantidad', 'energia']].corr()

# Imprime la matriz de correlación
print("Matriz de Correlación:")
print(matriz_correlacion)


Matriz de Correlación:
          cantidad   energia
cantidad  1.000000  0.832692
energia   0.832692  1.000000


In [67]:
# ########################################
data_agrupada

,tarifa,cantidad,energia,total
0,Comercial,1404,5.311848e+04,1.943426e+08
1,Industrial/Comercial Grande,11176,5.573125e+04,1.943426e+08
2,Mercado a Término,2,1.979000e-03,1.943426e+08
3,Residencial,8247,8.529875e+04,1.943426e+08
4,TOT,20829,1.941485e+08,1.943426e+08


In [73]:
# Calcular los coeficientes de correlación por tipo de tarifa entre las variables identificadas
correlaciones_por_tarifa = {}
for tarifa, grupo in grupo_tarifas:
    if 'energia' in grupo.columns:  # Verificar si la columna 'energia' existe en el grupo
        correlacion = grupo.corr().loc.get('cantidad', {}).get('energia', None)
        if correlacion is not None:
            correlaciones_por_tarifa[tarifa] = correlacion

# Imprimir las correlaciones por tipo de tarifa
print("Coeficientes de Correlación por Tipo de Tarifa:")
for tarifa, correlacion in correlaciones_por_tarifa.items():
    print(f"{tarifa}: {correlacion}")


Coeficientes de Correlación por Tipo de Tarifa:


In [ ]:
# ########################################

In [72]:
# Paso 1: Filtrar el DataFrame para los grupos de tarifa de interés
grupos_de_interes = ['Industriales', 'Comerciales', 'Residenciales']
data_filtrada = data_agrupada[data_agrupada['tarifa'].isin(grupos_de_interes)]

# Paso 2: Inicializar un diccionario para almacenar las matrices de correlación
matrices_correlacion = {}

# Paso 3: Calcular las matrices de correlación
for grupo in grupos_de_interes:
    # Filtrar los datos del grupo de tarifa actual
    datos_grupo = data_filtrada[data_filtrada['tarifa'] == grupo]
    
    # Calcular la matriz de correlación entre variables explicativas y variable objetivo
    matriz_correlacion = datos_grupo[['cantidad', 'energia']].corrwith(datos_grupo['total'])
    
    # Almacenar la matriz de correlación en el diccionario
    matrices_correlacion[grupo] = matriz_correlacion

# Imprimir las matrices de correlación
for grupo, matriz in matrices_correlacion.items():
    print(f"Matriz de Correlación para {grupo}:")
    print(matriz)


Matriz de Correlación para Industriales:
cantidad   NaN
energia    NaN
dtype: float64
Matriz de Correlación para Comerciales:
cantidad   NaN
energia    NaN
dtype: float64
Matriz de Correlación para Residenciales:
cantidad   NaN
energia    NaN
dtype: float64


In [71]:
# Resumen de los datos estadísticos de energía renovable por tipo de tecnologia.
# Cinco medidas descriptivas de los datos: el primer cuartil (Q1), la mediana (Q2), el tercer cuartil (Q3),
# el valor máximo, y el valor mínimo
data_agrupada.describe()

,cantidad,energia,total
count,5.000000,5.000000e+00,5.000000e+00
mean,8331.600000,3.886853e+07,1.943426e+08
std,8391.314754,8.680414e+07,0.000000e+00
min,2.000000,1.979000e-03,1.943426e+08
25%,1404.000000,5.311848e+04,1.943426e+08
50%,8247.000000,5.573125e+04,1.943426e+08
75%,11176.000000,8.529875e+04,1.943426e+08
max,20829.000000,1.941485e+08,1.943426e+08


In [76]:
# Calcular el coeficiente de correlación de Pearson
correlacion_pearson = data_agrupada[['cantidad', 'energia', 'total']].corr(method='pearson')

# Calcular el coeficiente de correlación de Spearman
correlacion_spearman = data_agrupada[['cantidad', 'energia', 'total']].corr(method='spearman')

# Calcular el coeficiente de correlación de Kendall
correlacion_kendall = data_agrupada[['cantidad', 'energia', 'total']].corr(method='kendall')

print("Coeficiente de Correlación de Pearson:")
print(correlacion_pearson)

print("Coeficiente de Correlación de Spearman:")
print(correlacion_spearman)

print("Coeficiente de Correlación de Kendall:")
print(correlacion_kendall)


Coeficiente de Correlación de Pearson:
          cantidad   energia  total
cantidad  1.000000  0.832692    NaN
energia   0.832692  1.000000    NaN
total          NaN       NaN    NaN
Coeficiente de Correlación de Spearman:
          cantidad  energia  total
cantidad       1.0      0.9    NaN
energia        0.9      1.0    NaN
total          NaN      NaN    NaN
Coeficiente de Correlación de Kendall:
          cantidad  energia  total
cantidad       1.0      0.8    NaN
energia        0.8      1.0    NaN
total          NaN      NaN    1.0


In [77]:
corr_spearman = []

for i, fila in data_agrupada.iterrows():
    tari_calc = data_agrupada[['cantidad', 'energia', 'total']].corr(method='spearman')
    corr_spearman.append(tari_calc)

# Convierte las filas calculadas en un nuevo DataFrame
#df_spearman = pd.DataFrame(corr_spearman)
corr_spearman

[          cantidad  energia  total
 cantidad       1.0      0.9    NaN
 energia        0.9      1.0    NaN
 total          NaN      NaN    NaN,
           cantidad  energia  total
 cantidad       1.0      0.9    NaN
 energia        0.9      1.0    NaN
 total          NaN      NaN    NaN,
           cantidad  energia  total
 cantidad       1.0      0.9    NaN
 energia        0.9      1.0    NaN
 total          NaN      NaN    NaN,
           cantidad  energia  total
 cantidad       1.0      0.9    NaN
 energia        0.9      1.0    NaN
 total          NaN      NaN    NaN,
           cantidad  energia  total
 cantidad       1.0      0.9    NaN
 energia        0.9      1.0    NaN
 total          NaN      NaN    NaN]

In [78]:
# Agrupar los datos por 'tarifa'
grupos = data_agrupada.groupby('tarifa')

# Inicializar listas para almacenar las correlaciones de Spearman por tarifa
tarifas = []
correlaciones_spearman = []

# Calcular la correlación de Spearman para cada grupo (tarifa)
for tarifa, grupo in grupos:
    correlacion_spearman = stats.spearmanr(grupo['energia'], grupo['total'])
    
    tarifas.append(tarifa)
    correlaciones_spearman.append(correlacion_spearman.correlation)

# Crear un DataFrame con las correlaciones de Spearman
correlaciones_df = pd.DataFrame({'tarifa': tarifas, 'correlacion_spearman': correlaciones_spearman})

print(correlaciones_df)


                        tarifa  correlacion_spearman
0                    Comercial                   NaN
1  Industrial/Comercial Grande                   NaN
2            Mercado a Término                   NaN
3                  Residencial                   NaN
4                          TOT                   NaN


In [79]:
# Agrupar los datos por 'tarifa'
grupos = data_agrupada.groupby('tarifa')

# Inicializar listas para almacenar las correlaciones de Pearson por tecnología
tarifas = []
correlaciones_pearson = []

# Calcular la correlación de Pearson para cada grupo (tarifas)
for tarifa, grupo in grupos:
    correlacion_pearson = grupo['cantidad'].corr(grupo['total'], method='pearson')
    
    tarifas.append(tarifa)
    correlaciones_pearson.append(correlacion_pearson)

# Crear un DataFrame con las correlaciones de Pearson
correlaciones_df = pd.DataFrame({'tarifa': tarifas, 'correlacion_pearson': correlaciones_pearson})

print(correlaciones_df)


                        tarifa  correlacion_pearson
0                    Comercial                  NaN
1  Industrial/Comercial Grande                  NaN
2            Mercado a Término                  NaN
3                  Residencial                  NaN
4                          TOT                  NaN


C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


In [80]:
# Agrupar los datos por 'tarifa'
grupos = data_agrupada.groupby('tarifa')

# Inicializar listas para almacenar las correlaciones de Kendall por tecnología
tarifas = []
correlaciones_kendall = []

# Calcular la correlación de Kendall para cada grupo (tarifa)
for tarifa, grupo in grupos:
    correlacion_kendall, _ = kendalltau(grupo['cantidad'], grupo['total'])
    
    tarifas.append(tarifa)
    correlaciones_kendall.append(correlacion_kendall)

# Crear un DataFrame con las correlaciones de Kendall
correlaciones_df = pd.DataFrame({'tarifa': tarifas, 'correlacion_kendall': correlaciones_kendall})

print(correlaciones_df)


                        tarifa  correlacion_kendall
0                    Comercial                  NaN
1  Industrial/Comercial Grande                  NaN
2            Mercado a Término                  NaN
3                  Residencial                  NaN
4                          TOT                  NaN


In [ ]:
data_agrupada

In [82]:

# Supongamos que tienes un DataFrame llamado 'data' con las características (X) y las etiquetas (y)
X = data_agrupada[['cantidad', 'energia']]
y = data_agrupada['total']

# Dividir los datos en conjuntos de entrenamiento y prueba (por ejemplo, 80% entrenamiento y 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train y y_train son los datos de entrenamiento
# X_test y y_test son los datos de prueba


In [83]:
# Crear un modelo de regresión lineal
modelo = LinearRegression()

# Entrenar el modelo en los datos de entrenamiento
modelo.fit(X_train, y_train)

# Realizar predicciones en los datos de prueba
y_pred = modelo.predict(X_test)

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)

# Imprimir el valor de R^2
print("Coeficiente de determinación (R^2):", r2)


Coeficiente de determinación (R^2): nan


C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
